# Creating single images

To run a YOLO model with transfer learning (just for classification, without bounding boxes for now), we're going to need to create a dataset for classification of the following format:
- Folder directory (containing training data)
    - Class 1 (tornado)
        - Many images of tornadoes (each file one image)
    - Class 2 (not tornado)
        - Many images of not tornadoes (each file one image)
     
Just as a test, we're trying to create some single images (with only one channel and lower tilt) just to see if those
do any better than the full set of all 12 images at once concatenated together.

In [7]:
import os
import subprocess
import sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from tornet.data.loader import TornadoDataLoader
from tornet.display.display import plot_radar
from tornet.data.constants import ALL_VARIABLES

plt.ioff()

TORNET_DATA_INPUT_FOLDER = "/mnt/c/users/handypark/Documents/Grad_School_Courses/CS_230/tornet"
catalog_path = os.path.join(TORNET_DATA_INPUT_FOLDER,'catalog.csv')
data_type='train'
years = [2014, 2015]

catalog = pd.read_csv(catalog_path, parse_dates=['start_time','end_time'])
catalog = catalog[catalog['type'] == data_type]
catalog = catalog[catalog.start_time.dt.year.isin(years)]
catalog = catalog.sample(frac=1, random_state=5678)

catalog_tornadoes = catalog[catalog.ef_number != -1]
catalog_not_tornadoes = catalog[catalog.ef_number == -1]

tornado_list = [os.path.join(TORNET_DATA_INPUT_FOLDER, f) for f in catalog_tornadoes.filename]
not_tornado_list = [os.path.join(TORNET_DATA_INPUT_FOLDER, f) for f in catalog_not_tornadoes.filename]

#vars_to_plot=['DBZ', 'VEL', 'KDP', 'RHOHV', 'ZDR', 'WIDTH']

vars_to_plot = ['VEL']
n_frames=1
tilt_last=True
D_ROOT = "/mnt/d/tornetExamples"

tornado_data = TornadoDataLoader(tornado_list, variables=vars_to_plot, n_frames=n_frames)
not_tornado_data = TornadoDataLoader(not_tornado_list, variables=vars_to_plot, n_frames=n_frames)

for didx in range(len(tornado_data)): #range(len(tornado_data)):
    x = tornado_data[didx]

    fig = plt.figure(figsize=(3,3),edgecolor='k')
    plot_radar(x,
               fig=fig,
               channels=vars_to_plot,
               include_cbar=True,
               time_idx=-1, 
               n_rows=1, n_cols=1)
    plt.savefig('{}/vel_2014_and_2015/{}/tornado/fig_VEL_{}.png'.format(D_ROOT, data_type, didx))
    plt.close() 

for didx in range(len(not_tornado_data)): #range(len(tornado_data)):
    x = not_tornado_data[didx]

    fig = plt.figure(figsize=(3,3),edgecolor='k')
    plot_radar(x,
               fig=fig,
               channels=vars_to_plot,
               include_cbar=True,
               time_idx=-1, 
               n_rows=1, n_cols=1)
    plt.savefig('{}/vel_2014_and_2015/{}/not_tornado/fig_VEL_{}.png'.format(D_ROOT, data_type, didx))
    plt.close() 